In [1]:
from pystac_client import Client
from pystac.extensions.eo import EOExtension as eo
import planetary_computer as pc
import geopandas as gpd
import rasterio
from rasterio import warp, windows


In [2]:
def fetch_s2_hrefs(catalog, aoi, time_range, max_nodata=20, max_cloud=.5):
    search = catalog.search(
        collections=["sentinel-2-l2a"],
        intersects=aoi,
        datetime=time_range,
        query={
            "s2:nodata_pixel_percentage": {"lt": max_nodata},
            "s2:high_proba_clouds_percentage": {"lt": max_cloud}
        }
    )

    # for each item, get hrefs to each band
    items = search.get_all_items()
    links, properties = {}, {}
    for item in items:
        print(item)
        bands = {}
        for k, v in item.assets.items():
            if k.startswith("B") or k == "SCL":
                bands[k] = pc.sign(v.href)
        id = item.properties["s2:product_uri"]
        properties[id] = item.properties
        links[id] = bands

    return properties, links


def write_scenes(scenes, aoi):
    for id, bands in scenes.items():
        write_scene(id, bands, aoi)


from rasterio import windows
from rasterio.mask import mask

def write_scene(id, bands, aoi):
    one_band = next(iter(bands.values()))
    scene = rasterio.open(one_band)
    masked, transform = mask(scene, [aoi], crop=True)

    meta = scene.meta
    meta["count"] = len(bands)
    meta["transform"] = transform
    meta["width"] = masked.shape[2]
    meta["height"] = masked.shape[1]
    
    window = windows.from_bounds(*aoi.bounds, transform=meta["transform"])
    with rasterio.open(f"{id}.tif", "w", **meta) as dst:
        for i, (k, v) in enumerate(bands.items()):
            scene = rasterio.open(v)
            masked, _ = mask(scene, [aoi], crop=True)
            dst.write_band(i + 1, masked.squeeze())

In [3]:
import pandas as pd

catalog = Client.open("https://planetarycomputer.microsoft.com/api/stac/v1")
lakes = gpd.read_file("../data/GL_3basins_2015.shp")
lakes = lakes[lakes.Area > 1].reset_index()
lakes_proj = lakes.to_crs(32645)

properties = []
all_links = []
for i in range(len(lakes)):
    print(i)
    try:
        properties_, links = fetch_s2_hrefs(catalog, lakes.geometry[i], "2015-01-01/2022-01-01")
        all_links.append(links)
    except:
        continue
    #aoi_proj = lakes_proj.geometry[i].buffer(500).envelope
    #write_scenes(links, aoi_proj)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20


ValueError: No objects to concatenate

In [4]:
all_links

[{},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {}]

In [ ]:
!gdalwarp -s_srs EPSG:32645 -t_srs EPSG:4326 S2A_MSIL2A_20191015T044751_N0212_R076_T45RVN_20201004T022821.SAFE.tif out.tif

In [5]:
search = catalog.search(
    collections=["sentinel-2-l2a"],
    intersects=lakes.geometry[0].bounds,
    datetime="2021-01-01/2022-01-01",
    query={
        "s2:nodata_pixel_percentage": {"lt": 20},
        "s2:high_proba_clouds_percentage": {"lt": 0.5}
    }
)

In [6]:
next(search.get_items())

APIError: 7 validation errors for Request
body -> intersects
  value is not a valid dict (type=type_error.dict)
body -> intersects
  value is not a valid dict (type=type_error.dict)
body -> intersects
  value is not a valid dict (type=type_error.dict)
body -> intersects
  value is not a valid dict (type=type_error.dict)
body -> intersects
  value is not a valid dict (type=type_error.dict)
body -> intersects
  value is not a valid dict (type=type_error.dict)
body -> intersects
  value is not a valid dict (type=type_error.dict)